In [7]:
%connect_info

{
  "stdin_port": 64014,
  "signature_scheme": "hmac-sha256",
  "iopub_port": 64013,
  "key": "90c695d8-b499-4623-98fa-61ced824c30f",
  "control_port": 64015,
  "hb_port": 64016,
  "shell_port": 64012,
  "kernel_name": "",
  "transport": "tcp",
  "ip": "127.0.0.1"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-d43fbded-6e79-42da-a17d-08aaf13fc1de.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [8]:
from jupyter_client import KernelManager
from jupyter_client import KernelClient, BlockingKernelClient
from jupyter_client.connect import find_connection_file
from jupyter_core.paths import jupyter_data_dir, jupyter_runtime_dir
from queue import Empty
import os
import glob
import time

In [9]:
def flush_channels(kc):
    channels = [kc.shell_channel, kc.iopub_channel]
    msgs = []
    while True:
        for channel in channels:
            try:
                msg = channel.get_msg(block=True, timeout=1)
            except Empty:
                channels.remove(channel)
            else:
                print(msg["msg_type"], msg["content"], "\n")
        
        if not channels:
            break

In [10]:
def get_connection_files():
    return(glob.glob(os.path.join(jupyter_runtime_dir(), "kernel-*.json")))

def active_connection_files():
    clients = []
    cfs = get_connection_files()
    for cf in cfs:
        kc = BlockingKernelClient(connection_file=cf)
        clients.append(kc)
        kc.load_connection_file()
        kc.start_channels()

    time.sleep(1.1)
    active_cfs = [kc.connection_file for kc in clients if kc.hb_channel.is_beating()]
    for kc in clients:
        kc.stop_channels()
        
    return(active_cfs)

In [11]:
active_connection_files()

ZMQError: Too many open files

In [46]:
cf = find_connection_file("1168db12")
kc = BlockingKernelClient(connection_file=cf)
kc.load_connection_file()
kc.start_channels()

In [47]:
kc.hb_channel.is_beating()

True

In [101]:
kc.kernel_info()
kc.shell_channel.get_msg(block=True, timeout=0.5)
kc.iopub_channel.get_msg(block=True, timeout=0.5)

{'buffers': [],
 'content': {'execution_state': 'busy'},
 'header': {'date': datetime.datetime(2016, 5, 3, 18, 5, 59, 246523),
  'msg_id': 'e610438f-7f9f-409c-b7fb-6b69dba0d957',
  'msg_type': 'status',
  'session': '0bcb0fe4-2253-4099-be41-8b0e94ec37c9',
  'username': 'randy',
  'version': '5.0'},
 'metadata': {},
 'msg_id': 'e610438f-7f9f-409c-b7fb-6b69dba0d957',
 'msg_type': 'status',
 'parent_header': {'date': datetime.datetime(2016, 5, 3, 18, 5, 59, 244933),
  'msg_id': '44c1f82a-1a80-4325-8a6e-b9facbe779c6',
  'msg_type': 'kernel_info_request',
  'session': '920c9636-2245-42af-ab8d-cd8cdb55b9ae',
  'username': 'randy',
  'version': '5.0'}}

In [102]:
flush_channels(kc)

status {'execution_state': 'idle'} 

